# Notebook to test the Bedrock Agents and UC Functions.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
 %reload_ext autoreload

In [3]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration
from unitycatalog.ai.bedrock.toolkit import UCFunctionToolkit


In [4]:
config = Configuration()
config.host = "http://localhost:8080/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

CATALOG = "AICatalog1"
SCHEMA = "AISchema1"


In [ ]:
# Sample function to get weather in celsius
    
def get_weather_in_celsius(location_id: str, fetch_date: str) -> str:
    """Test function for AWS Bedrock integration.

    Args:
        location_id (str): The name to be included in the greeting message.
        fetch_date (str): The date with the location 

    Raises:
        Exception: If there is an error during the function execution.

    Returns:
        str: Weather result.
    """
    try:
        # Fetch from Databricks SQL Warehouse based UC function execution 
        return "23"
    except Exception as e:
        raise Exception(f"Error occurred: {e}")

In [ ]:
def get_weather_in_fh(location_id: str, fetch_date: str) -> str:
    """Test function for AWS Bedrock integration.

    Args:
        location_id (str): The name to be included in the greeting message.
        fetch_date (str): The date with the location 

    Raises:
        Exception: If there is an error during the function execution.

    Returns:
        str: Weather result.
    """
    try:
        # Fetch from Databricks SQL Warehouse based UC function execution 
        return "72"
    except Exception as e:
        raise Exception(f"Error occurred: {e}")

In [ ]:
client.uc.create_catalog(name=CATALOG, comment="Catalog for AI functions")

In [ ]:
client.uc.create_schema(catalog_name=CATALOG, name=SCHEMA, comment="Schema for AI functions")

In [ ]:
client.create_python_function(
    func=get_weather_in_celsius, 
    catalog=CATALOG, 
    schema=SCHEMA, 
    replace=True)

In [ ]:
client.create_python_function(
    func=get_weather_in_fh, 
    catalog=CATALOG, 
    schema=SCHEMA, 
    replace=True)

## Check if UC function exists

In [64]:
function_name = f"{CATALOG}.{SCHEMA}.get_weather_in_celsius"

In [65]:
try:
    print(client.get_function(function_name))
except Exception as e:
    print(f"Function {function_name} does not exist, creating it...")

name='get_weather_in_celsius' catalog_name='AICatalog1' schema_name='AISchema1' input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location_id', type_text='STRING', type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.'), FunctionParameterInfo(name='fetch_date', type_text='STRING', type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date with the location"}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The date with the location')

In [5]:
#function_name = f"{CATALOG}.{SCHEMA}.get_weather_in_celsius"
function_name_c = f"{CATALOG}.{SCHEMA}.get_weather_in_celsius"
function_name_fh = f"{CATALOG}.{SCHEMA}.get_weather_in_fh"
toolkit = UCFunctionToolkit(function_names=[function_name_c, function_name_fh], client=client)

In [68]:
print(toolkit.client.get_function(function_name))

name='get_weather_in_celsius' catalog_name='AICatalog1' schema_name='AISchema1' input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location_id', type_text='STRING', type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.'), FunctionParameterInfo(name='fetch_date', type_text='STRING', type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date with the location"}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The date with the location')

In [6]:
import pprint
pprint.pp(toolkit.client.list_functions(catalog=CATALOG, schema=SCHEMA))

[FunctionInfo(name='get_weather_in_celsius', catalog_name='AICatalog1', schema_name='AISchema1', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location_id', type_text='STRING', type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.'), FunctionParameterInfo(name='fetch_date', type_text='STRING', type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date with the location"}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The date wi

# Adding Bedrock SDK calls 


In [7]:
import boto3, pprint, json, time, uuid

In [ ]:
#boto3.setup_default_session()

## Calling agent with a specific version or state by mentioning the agent alias. 
### Need to change the version or agent alis to test the changes to the agent or agent action groups/functions

In [8]:
agent_id = "HXPGL1POGD"
agent_alias_id = "4TL4EPM5HT"

In [73]:
# Bedrock agent configuration
session = toolkit.create_session(agent_id=agent_id,
                                agent_alias_id=agent_alias_id,
                                catalog_name=CATALOG,
                                schema_name=SCHEMA,
                                #function_name="get_weather_in_celsius"
                                )
# Generate unique session ID
session_id = str(uuid.uuid1())

Initialized BedrockSession with agent_id: HXPGL1POGD, agent_alias_id: 4TL4EPM5HT, catalog_name: AICatalog1 schema_name: AISchema1, 


In [74]:
client.get_function(f"{CATALOG}.{SCHEMA}.get_weather_in_celsius")

FunctionInfo(name='get_weather_in_celsius', catalog_name='AICatalog1', schema_name='AISchema1', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location_id', type_text='STRING', type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.'), FunctionParameterInfo(name='fetch_date', type_text='STRING', type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date with the location"}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The date wit

In [75]:
response = session.invoke_agent(
                input_text="What is the weather for location 1234 and date of 2024-11-19",
                enable_trace=True,
                session_id=session_id,
                uc_client = toolkit.client
                )

Response from invoke agent: {'ResponseMetadata': {'RequestId': '224a7c76-fec4-4151-91e2-b57fe7b0329d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 12 Feb 2025 01:19:33 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '224a7c76-fec4-4151-91e2-b57fe7b0329d', 'x-amz-bedrock-agent-session-id': '647ea11c-e8df-11ef-bc25-dae3f3b23e8c', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '647ea11c-e8df-11ef-bc25-dae3f3b23e8c', 'completion': <botocore.eventstream.EventStream object at 0x138c2db80>}
Tool Call Results: [{'function_name': 'uc-ai-bedrock-action-group-name__get_weather_in_celsius', 'parameters': {'fetch_date': '2024-11-19', 'location_id': '1234'}, 'invocation_id': '36b74a75-1e1c-4f87-a99c-73a9fd60b632-uc-result'}]
****************************
Tool Calls: uc-ai-bedrock-action-group-name__get_weather_in_celsius
*

In [76]:
import pprint 
pprint.pp(response)

BedrockToolResponse(raw_response={'ResponseMetadata': {'RequestId': '42632d9a-1638-4e5d-9457-353ae3fcafc7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 12 Feb 2025 01:20:42 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '42632d9a-1638-4e5d-9457-353ae3fcafc7', 'x-amz-bedrock-agent-session-id': '647ea11c-e8df-11ef-bc25-dae3f3b23e8c', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '647ea11c-e8df-11ef-bc25-dae3f3b23e8c', 'completion': <botocore.eventstream.EventStream object at 0x1392d84b0>}, tool_calls=[{'function_name': 'uc-ai-bedrock-action-group-name__get_weather_in_celsius', 'parameters': {'fetch_date': '2024-11-19', 'location_id': '1234'}, 'invocation_id': '36b74a75-1e1c-4f87-a99c-73a9fd60b632-uc-result'}], tool_results=[], response_body=None)


In [77]:

# Run the stream with wrapping
response.print_stream_with_wrapping(response.get_stream(), max_line_length=50)


The weather for location ID 1234 on 2024-11-19 is 
23°C (degrees Celsius).


## Fetch weather in Fahrenheit with a new agent alias 

In [ ]:
# Bedrock agent configuration
agent_id = "HXPGL1POGD"
agent_alias_id_fh = "UWOLYMYEIT"

session_fh = toolkit.create_session(agent_id=agent_id,
                                agent_alias_id=agent_alias_id_fh,
                                catalog_name=CATALOG,
                                schema_name=SCHEMA,
                                #function_name="get_weather_in_celsius"
                                )
# Generate unique session ID
session_id_fh = str(uuid.uuid1())

Initialized BedrockSession with agent_id: HXPGL1POGD, agent_alias_id: UWOLYMYEIT, catalog_name: AICatalog1 schema_name: AISchema1, 


In [15]:


response_fh = session_fh.invoke_agent(
                input_text="What is the weather in Fahrenheit for location 1234 and date of 2024-11-19?.",
                enable_trace=True,
                session_id=session_id_fh,
                uc_client = toolkit.client
                )


# Run the stream with wrapping
response_fh.print_stream_with_wrapping(response_fh.get_stream(), max_line_length=50)

Response from invoke agent: {'ResponseMetadata': {'RequestId': '76f7bcc2-772d-4cc0-a750-48698253ac6f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 12 Feb 2025 02:07:51 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '76f7bcc2-772d-4cc0-a750-48698253ac6f', 'x-amz-bedrock-agent-session-id': '29682254-e8e6-11ef-8661-dae3f3b23e8c', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '29682254-e8e6-11ef-8661-dae3f3b23e8c', 'completion': <botocore.eventstream.EventStream object at 0x1182627b0>}
Tool Call Results: [{'function_name': 'uc-ai-bedrock-action-group-name-fh__get_weather_in_fh', 'parameters': {'fetch_date': '2024-11-19', 'location_id': '1234'}, 'invocation_id': '5ad15e91-84ea-477f-948a-0fd662b17d08'}]
****************************
Tool Calls: uc-ai-bedrock-action-group-name-fh__get_weather_in_fh
***************

## Agent with two action groups for C and F with a new agent alias

In [12]:
# Bedrock agent configuration
agent_id = "HXPGL1POGD"
agent_alias_id_c_fh = "X1KRKIAVYT"

session_fh = toolkit.create_session(agent_id=agent_id,
                                agent_alias_id=agent_alias_id_c_fh,
                                catalog_name=CATALOG,
                                schema_name=SCHEMA,
                                #function_name="get_weather_in_celsius"
                                )
# Generate unique session ID
session_id_cfh = str(uuid.uuid1())

Initialized BedrockSession with agent_id: HXPGL1POGD, agent_alias_id: X1KRKIAVYT, catalog_name: AICatalog1 schema_name: AISchema1, 


In [18]:

response_fh_1 = session_fh.invoke_agent(
                input_text="What is the weather in Fahrenheit for location 1234 and date of 2024-11-19?.",
                enable_trace=True,
                session_id=session_id_cfh,
                uc_client = toolkit.client
                )


# Run the stream with wrapping
response_fh_1.print_stream_with_wrapping(response_fh_1.get_stream(), max_line_length=50)

Response from invoke agent: {'ResponseMetadata': {'RequestId': 'c6eb0f60-2635-47ef-bc95-e3060b851aef', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 12 Feb 2025 02:13:33 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'c6eb0f60-2635-47ef-bc95-e3060b851aef', 'x-amz-bedrock-agent-session-id': 'f07ffa06-e8e6-11ef-8661-dae3f3b23e8c', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': 'f07ffa06-e8e6-11ef-8661-dae3f3b23e8c', 'completion': <botocore.eventstream.EventStream object at 0x118c3c270>}
Tool Call Results: [{'function_name': 'uc-ai-bedrock-action-group-name-fh__get_weather_in_fh', 'parameters': {'fetch_date': '2024-11-19', 'location_id': '1234'}, 'invocation_id': 'd5a43861-7167-48d1-a3c8-c7f3163dd30d'}]
****************************
Tool Calls: uc-ai-bedrock-action-group-name-fh__get_weather_in_fh
***************

In [19]:
import time 

In [20]:
time.sleep(65)

In [21]:
response_c_1 = session_fh.invoke_agent(
                input_text="What is the weather in Centigrade for location 7890 and date of 2024-11-19?.",
                enable_trace=True,
                session_id=session_id_fh,
                uc_client = toolkit.client
                )


# Run the stream with wrapping
response_c_1.print_stream_with_wrapping(response_c_1.get_stream(), max_line_length=50)

Response from invoke agent: {'ResponseMetadata': {'RequestId': '6ed6fa49-8d35-4af4-89c1-3948cf196b35', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 12 Feb 2025 02:15:51 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '6ed6fa49-8d35-4af4-89c1-3948cf196b35', 'x-amz-bedrock-agent-session-id': '29682254-e8e6-11ef-8661-dae3f3b23e8c', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '29682254-e8e6-11ef-8661-dae3f3b23e8c', 'completion': <botocore.eventstream.EventStream object at 0x118ae6d50>}
Tool Call Results: [{'function_name': 'uc-ai-bedrock-action-group-name-c__get_weather_in_celsius', 'parameters': {'fetch_date': '2024-11-19', 'location_id': '7890'}, 'invocation_id': 'e7b69e92-57f7-42fb-ba0b-91d70f8dd2e3'}]
****************************
Tool Calls: uc-ai-bedrock-action-group-name-c__get_weather_in_celsius
*******

In [ ]:
## Multi action groups with single function call response. 

In [13]:
# Bedrock agent configuration
agent_id = "HXPGL1POGD"
agent_alias_id_cfh2 = "X1KRKIAVYT"

session_cfh_2 = toolkit.create_session(agent_id=agent_id,
                                agent_alias_id=agent_alias_id_cfh2,
                                catalog_name=CATALOG,
                                schema_name=SCHEMA,
                                #function_name="get_weather_in_celsius"
                                )
# Generate unique session ID
session_id_cfh_2 = str(uuid.uuid1())

Initialized BedrockSession with agent_id: HXPGL1POGD, agent_alias_id: X1KRKIAVYT, catalog_name: AICatalog1 schema_name: AISchema1, 


In [14]:
response_cfh_2 = session_cfh_2.invoke_agent(
                input_text="What is the weather in Centigrade and Fahrenheit for location 1234 and date of 2024-11-19?.",
                enable_trace=True,
                session_id=session_id_cfh_2,
                uc_client = toolkit.client
                )


# Run the stream with wrapping
response_cfh_2.print_stream_with_wrapping(response_cfh_2.get_stream(), max_line_length=50)

Response from invoke agent: {'ResponseMetadata': {'RequestId': 'dbdc9205-a2a1-4895-b7d4-16dbc90913be', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 12 Feb 2025 17:36:46 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'dbdc9205-a2a1-4895-b7d4-16dbc90913be', 'x-amz-bedrock-agent-session-id': 'ec10ac5a-e967-11ef-acf1-dae3f3b23e8c', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': 'ec10ac5a-e967-11ef-acf1-dae3f3b23e8c', 'completion': <botocore.eventstream.EventStream object at 0x117982210>}
Tool Call Results: [{'function_name': 'uc-ai-bedrock-action-group-name-c__get_weather_in_celsius', 'parameters': {'fetch_date': '2024-11-19', 'location_id': '1234'}, 'invocation_id': 'd9f3c992-c3b3-484b-afdd-18c08c94b5cb'}]
****************************
Tool Calls: uc-ai-bedrock-action-group-name-c__get_weather_in_celsius
*******

## Single action group multiple functions call response. 

In [8]:
# Bedrock agent configuration
agent_id = "HXPGL1POGD"
agent_alias_id_s_ag_2fns = "TXKIZ3FMBB"

session_s_ag_2fns = toolkit.create_session(agent_id=agent_id,
                                agent_alias_id=agent_alias_id_s_ag_2fns,
                                catalog_name=CATALOG,
                                schema_name=SCHEMA,
                                #function_name="get_weather_in_celsius"
                                )
# Generate unique session ID
session_id_s_ag_2fns = str(uuid.uuid1())

Initialized BedrockSession with agent_id: HXPGL1POGD, agent_alias_id: TXKIZ3FMBB, catalog_name: AICatalog1 schema_name: AISchema1, 


In [ ]:
import time 
time.sleep(65)

In [ ]:
response_s_ag_2fns = session_s_ag_2fns.invoke_agent(
                input_text="What is the weather in Centigrade and Fahrenheit for location 7890 and date of 2024-11-19?.",
                enable_trace=True,
                session_id=session_id_s_ag_2fns,
                uc_client = toolkit.client
                )




Response from invoke agent: {'ResponseMetadata': {'RequestId': '3fa5245f-fea6-4740-9e19-9febd569c8c3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 12 Feb 2025 17:14:31 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '3fa5245f-fea6-4740-9e19-9febd569c8c3', 'x-amz-bedrock-agent-session-id': 'c5aea736-e964-11ef-acf1-dae3f3b23e8c', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': 'c5aea736-e964-11ef-acf1-dae3f3b23e8c', 'completion': <botocore.eventstream.EventStream object at 0x11716f380>}
Tool Call Results: [{'function_name': 'uc-ai-bedrock-action-group-name-cfh__get_weather_in_celsius', 'parameters': {'fetch_date': '2024-11-19', 'location_id': '7890'}, 'invocation_id': 'fa666a34-d23d-4b42-9eaa-6866af4c2dfe'}]
****************************
Tool Calls: uc-ai-bedrock-action-group-name-cfh__get_weather_in_celsius
***

NameError: name 'response_c_1' is not defined

In [11]:

# Run the stream with wrapping
response_s_ag_2fns.print_stream_with_wrapping(response_s_ag_2fns.get_stream(), max_line_length=50)